In [24]:
import numpy as np
import pandas as pd

from glob import glob
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
obdir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/'

#### Use the automated data as baseline 'truth'


In [133]:
site_auto = 'CLN_AUTO'
f_auto = glob(obdir + '%s*.csv'%site_auto)[0]

# Remove the units row (SI - C, mm)
data_auto = pd.read_csv(f_auto, low_memory=False)[1:]
data_auto = data_auto.drop(columns='Station_ID')

renames_auto = {'Date_Time':'time', 
           'air_temp_set_1':'tsfc_c', 
           'snow_interval_set_1':'snow_mm', 
           'snow_depth_set_1':'depth_mm', 
           'precip_accum_one_hour_set_1':'swe_mm'}

data_auto = data_auto.rename(columns=renames_auto)

# Fix the datatypes
data_auto['time'] = data_auto['time'].astype('datetime64[ns]')
data_auto = data_auto.set_index('time')

data_auto = data_auto.astype({k:np.float32 for k in data_auto.keys()})

data_auto[:10]

,tsfc_c,snow_mm,depth_mm,swe_mm
time,,,,
1997-01-01 00:00:00,1.11,NaN,2311.399902,NaN
1997-01-01 01:00:00,0.56,NaN,2311.399902,NaN
1997-01-01 02:00:00,0.56,NaN,2311.399902,NaN
1997-01-01 03:00:00,2.22,NaN,2311.399902,NaN
1997-01-01 04:00:00,3.33,NaN,2311.399902,NaN
1997-01-01 05:00:00,3.89,NaN,2311.399902,NaN
1997-01-01 06:00:00,3.89,NaN,2311.399902,NaN
1997-01-01 07:00:00,2.78,NaN,2286.000000,NaN
1997-01-01 08:00:00,2.22,NaN,2286.000000,NaN


In [210]:
site = 'CLN'
f = glob(obdir + '%s*.csv'%site)[0]

data_raw = pd.read_csv(f, low_memory=False)
data_raw = data_raw.set_index(['DATE'])

data_time = np.array([[datetime.strptime(d+t, '%m/%d/%y %H%M') for t in [' 0400', ' 1600']] for d in data_raw.index]).flatten()
data_raw = np.array([[(data_raw.loc[d]['%sWATER'%t], data_raw.loc[d]['%sSNOW'%t]) for t in ['0400', '1600']] for d in data_raw.index])
data_raw = data_raw.reshape(-1, 2)

data = pd.DataFrame([data_time, data_raw[:, 0], data_raw[:, 1]]).T
data = data.rename(columns={0:'time', 1:'swe_in', 2:'snow_in'}).set_index('time')
data[:10]

,swe_in,snow_in
time,,
1999-11-01 04:00:00,0,0
1999-11-01 16:00:00,0,0
1999-11-02 04:00:00,0,0
1999-11-02 16:00:00,0,0
1999-11-03 04:00:00,0,0
1999-11-03 16:00:00,0,0
1999-11-04 04:00:00,0,0
1999-11-04 16:00:00,0,0
1999-11-05 04:00:00,0,0


#### Resample the hourlies to match the 12h

In [225]:
data_auto_swe12 = data_auto['swe_mm'].resample('12H', closed='right', label='right', base=4).sum()
data_auto_snow12 = data_auto['snow_mm'].resample('12H', closed='right', label='right', base=4).sum()
data_auto_12 = pd.DataFrame([data_auto_swe12, data_auto_snow12]).T
data_auto_12[:10]

,swe_mm,snow_mm
time,,
1997-01-01 04:00:00,0.0,0.0
1997-01-01 16:00:00,0.0,0.0
1997-01-02 04:00:00,0.0,0.0
1997-01-02 16:00:00,0.0,0.0
1997-01-03 04:00:00,0.0,0.0
1997-01-03 16:00:00,0.0,0.0
1997-01-04 04:00:00,0.0,0.0
1997-01-04 16:00:00,0.0,0.0
1997-01-05 04:00:00,0.0,0.0


In [231]:
data_all = data.merge(data_auto_12, on='time')
data_all[:10]

,swe_in,snow_in,swe_mm,snow_mm
time,,,,
1999-11-01 04:00:00,0,0,0.0,0.0
1999-11-01 16:00:00,0,0,0.0,0.0
1999-11-02 04:00:00,0,0,0.0,0.0
1999-11-02 16:00:00,0,0,0.0,0.0
1999-11-03 04:00:00,0,0,0.0,0.0
1999-11-03 16:00:00,0,0,0.0,0.0
1999-11-04 04:00:00,0,0,0.0,0.0
1999-11-04 16:00:00,0,0,0.0,0.0
1999-11-05 04:00:00,0,0,0.0,0.0


In [ ]:
# Go back and relabel these for man/auto